In [75]:
import pandas as pd
import numpy as np

from requests import get
from bs4 import BeautifulSoup
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests
import re
import time
import unicodedata
import markdown
import nltk

from env import github_token, github_username
from importlib import reload

# Acquire

In [2]:
"""
A module for obtaining repo readme and language data from the github API.
Before using this module, read through it, and follow the instructions marked
TODO.
After doing so, run it like this:
    python acquire.py
To create the `data.json` file that contains the data.
"""

# TODO: Make a github personal access token.
#     1. Go here and generate a personal access token: https://github.com/settings/tokens
#        You do _not_ need select any scopes, i.e. leave all the checkboxes unchecked
#     2. Save it in your env.py file under the variable `github_token`
# TODO: Add your github username to your env.py file under the variable `github_username`
# TODO: Add more repositories to the `REPOS` list below.

REPOS = [
    "gocodeup/codeup-setup-script",
    "gocodeup/movies-application",
    "torvalds/linux",
]

headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

if headers["Authorization"] == "token " or headers["User-Agent"] == "":
    raise Exception(
        "You need to follow the instructions marked TODO in this script before trying to use it"
    )


def github_api_request(url: str) -> Union[List, Dict]:
    response = requests.get(url, headers=headers)
    response_data = response.json()
    if response.status_code != 200:
        raise Exception(
            f"Error response from github api! status code: {response.status_code}, "
            f"response: {json.dumps(response_data)}"
        )
    return response_data


def get_repo_language(repo: str) -> str:
    url = f"https://api.github.com/repos/{repo}"
    repo_info = github_api_request(url)
    if type(repo_info) is dict:
        repo_info = cast(Dict, repo_info)
        if "language" not in repo_info:
            raise Exception(
                "'language' key not round in response\n{}".format(json.dumps(repo_info))
            )
        return repo_info["language"]
    raise Exception(
        f"Expecting a dictionary response from {url}, instead got {json.dumps(repo_info)}"
    )


def get_repo_contents(repo: str) -> List[Dict[str, str]]:
    url = f"https://api.github.com/repos/{repo}/contents/"
    contents = github_api_request(url)
    if type(contents) is list:
        contents = cast(List, contents)
        return contents
    raise Exception(
        f"Expecting a list response from {url}, instead got {json.dumps(contents)}"
    )


def get_readme_download_url(files: List[Dict[str, str]]) -> str:
    """
    Takes in a response from the github api that lists the files in a repo and
    returns the url that can be used to download the repo's README file.
    """
    for file in files:
        if file["name"].lower().startswith("readme"):
            return file["download_url"]
    return ""


def process_repo(repo: str) -> Dict[str, str]:
    """
    Takes a repo name like "gocodeup/codeup-setup-script" and returns a
    dictionary with the language of the repo and the readme contents.
    """
    contents = get_repo_contents(repo)
    readme_download_url = get_readme_download_url(contents)
    if readme_download_url == "":
        readme_contents = ""
    else:
        readme_contents = requests.get(readme_download_url).text
    return {
        "repo": repo,
        "language": get_repo_language(repo),
        "readme_contents": readme_contents,
    }


def scrape_github_data() -> List[Dict[str, str]]:
    """
    Loop through all of the repos and process them. Returns the processed data.
    """
    return [process_repo(repo) for repo in REPOS]


if __name__ == "__main__":
    data = scrape_github_data()
    json.dump(data, open("data.json", "w"), indent=1)

In [4]:
#url = "https://github.com/topics/minecraft"
reqs = requests.get(url)
soup = BeautifulSoup(reqs.text, 'html.parser')
urls = []
for link in soup.find_all('a',class_="v-align-middle"):
    print(link.get('href'))

In [ ]:
soup.find_all('div', class_="d-flex flex-1")

In [21]:
list_of_div_elements = soup.find_all('a', class_="data-hydro-click")
list_of_div_elements

[]

In [42]:
s = soup.find_all('h3', class_='f3 color-fg-muted text-normal lh-condensed')

In [54]:
s[0].find('a', class_="text-bold wb-break-word")['href']

'fogleman/Craft'

In [57]:
links =[]
all_minecraft_repos = soup.find_all('h3', class_='f3 color-fg-muted text-normal lh-condensed')
for each_minecraft_repo in all_minecraft_repos:
    #href_link = each_minecraft_repo.a.attrs["href"]
    href_link = each_minecraft_repo.find('a', class_="text-bold wb-break-word")['href'][1:]
    links.append(href_link)

In [58]:
links

['fogleman/Craft',
 'dnschneid/crouton',
 'PaperMC/Paper',
 'IdreesInc/Monocraft',
 'MinecraftForge/MinecraftForge',
 'nuno-faria/tiler',
 'cabaletta/baritone',
 'huanghongxun/HMCL',
 'cuberite/cuberite',
 'CaffeineMC/sodium-fabric',
 'MultiMC/Launcher',
 'GeyserMC/Geyser',
 'PojavLauncherTeam/PojavLauncher',
 'PrismarineJS/mineflayer',
 'pmmp/PocketMine-MP',
 'PrismLauncher/PrismLauncher',
 'EngineHub/WorldEdit',
 'IrisShaders/Iris',
 'feather-rs/feather',
 'Querz/mcaselector']

In [60]:
soup.find_all('a', class_="v-align-middle")

[]

In [92]:
url = 'https://github.com/search?3&q=minecraft&type=Repositories'

In [93]:
url

'https://github.com/search?3&q=minecraft&type=Repositories'

In [94]:
def get_minecraft_urls(url):
    reqs = requests.get(url)
    soup = BeautifulSoup(reqs.text, 'html.parser')
    urls = []
    for link in soup.find_all('a',class_="v-align-middle"):
        urls.append(link.get('href'))
    return urls

In [95]:
urls = get_minecraft_urls(url)
urls

['/MinecraftForge/MinecraftForge',
 '/fogleman/Minecraft',
 '/itzg/docker-minecraft-server',
 '/overviewer/Minecraft-Overviewer',
 '/TheGreyGhost/MinecraftByExample',
 '/minecraft-dev/MinecraftDev',
 '/jdah/minecraft-weekend',
 '/mouse0w0/MinecraftDeveloperGuide',
 '/Bukkit/Bukkit',
 '/TheRemote/MinecraftBedrockServer']

In [75]:
df_url = pd.DataFrame(urls, columns=['urls'])

In [77]:
#turn to csv
df_url.to_csv('df_urls.csv')

In [98]:
for i in range(3,53):
    url = f'https://github.com/search?{i}&q=minecraft&type=Repositories'
    url_list = []
    urls = get_minecraft_urls(url)
    url_list.append(urls)

In [100]:
url_list

[[]]

In [115]:
df.head()

,url
3,TheRemote/MinecraftBedrockServer
4,TheRemote/MinecraftBedrockServer


In [111]:
len(urls)

20

In [137]:
urls = []
for i in range(1,152):
    url = f'https://github.com/search?p={i}&q=minecraft&type=Repositories'
    reqs = requests.get(url)
    soup = BeautifulSoup(reqs.text, 'html.parser')
    time.sleep(5)
    
    for link in soup.find_all('a',class_="v-align-middle"):
        hyperlink = re.sub(r'/','', link.get('href'), count = 1)
        urls.append(hyperlink)
        time.sleep(1)

In [139]:
len(urls)

560

In [130]:
urls

['PrismarineJS/node-minecraft-protocol',
 'SpongePowered/SpongeAPI',
 'PrismarineJS/minecraft-data',
 'xPaw/PHP-Minecraft-Query',
 'samhogan/Minecraft-Unity3D',
 'GeyserMC/Geyser',
 'Hawstein/PyMinecraft',
 'jkutner/heroku-buildpack-minecraft',
 'walterhiggins/ScriptCraft',
 'doctorray117/minecraft-ondemand',
 'Ahtenus/minecraft-init',
 'Pierce01/MinecraftLauncher-core',
 'itzg/docker-minecraft-bedrock-server',
 'itsFrank/MinecraftHDL',
 'docker-archive/minecraft',
 'ldtteam/minecolonies',
 'YouHaveTrouble/minecraft-optimization',
 'dan200/ComputerCraft',
 'smith-j-travis/MinecraftAutoClicker',
 'toolbox4minecraft/amidst',
 'ReplayMod/ReplayMod',
 'pdinklag/MinecraftStats',
 'Wurst-Imperium/Wurst7',
 'MultiMC/Launcher',
 'pmmp/PocketMine-MP',
 'jdah/minecraft-again',
 'clear-code-projects/Minecraft-in-Python',
 'danba340/minecraft-freecodecamp',
 'LuckPerms/LuckPerms',
 'Advanced-Rocketry/AdvancedRocketry',
 'Minecraft-in-python/Minecraft',
 'overshard/docker-minecraft',
 'Fenixin/Mine

In [140]:
df2 = pd.DataFrame(columns=['repo_link'])
for i in range(len(urls)):
    df2.loc[i] = urls[i]

In [141]:
df2['repo_link'].nunique(), df.shape[0]

(556, 300)

In [6]:
#df = pd.read_csv(r'df_urls.csv', index_col=[0])

In [7]:
#df

,urls
0,/MinecraftForge/MinecraftForge
1,/fogleman/Minecraft
2,/itzg/docker-minecraft-server
3,/overviewer/Minecraft-Overviewer
4,/TheGreyGhost/MinecraftByExample
5,/minecraft-dev/MinecraftDev
6,/jdah/minecraft-weekend
7,/mouse0w0/MinecraftDeveloperGuide
8,/Bukkit/Bukkit
9,/TheRemote/MinecraftBedrockServer


In [3]:
url = f'https://github.com/search?q=minecraft'

In [8]:
df['urls'] = df['urls'].str.replace('/','',n=1)
df

,urls
0,MinecraftForge/MinecraftForge
1,fogleman/Minecraft
2,itzg/docker-minecraft-server
3,overviewer/Minecraft-Overviewer
4,TheGreyGhost/MinecraftByExample
5,minecraft-dev/MinecraftDev
6,jdah/minecraft-weekend
7,mouse0w0/MinecraftDeveloperGuide
8,Bukkit/Bukkit
9,TheRemote/MinecraftBedrockServer


In [9]:
"""
A module for obtaining repo readme and language data from the github API.
Before using this module, read through it, and follow the instructions marked
TODO.
After doing so, run it like this:
    python acquire.py
To create the `data.json` file that contains the data.
"""

# TODO: Make a github personal access token.
#     1. Go here and generate a personal access token: https://github.com/settings/tokens
#        You do _not_ need select any scopes, i.e. leave all the checkboxes unchecked
#     2. Save it in your env.py file under the variable `github_token`
# TODO: Add your github username to your env.py file under the variable `github_username`
# TODO: Add more repositories to the `REPOS` list below.

REPOS = list(df['urls'])

headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

if headers["Authorization"] == "token " or headers["User-Agent"] == "":
    raise Exception(
        "You need to follow the instructions marked TODO in this script before trying to use it"
    )


def github_api_request(url: str) -> Union[List, Dict]:
    response = requests.get(url, headers=headers)
    response_data = response.json()
    if response.status_code != 200:
        raise Exception(
            f"Error response from github api! status code: {response.status_code}, "
            f"response: {json.dumps(response_data)}"
        )
    return response_data


def get_repo_language(repo: str) -> str:
    url = f"https://api.github.com/repos/{repo}"
    repo_info = github_api_request(url)
    if type(repo_info) is dict:
        repo_info = cast(Dict, repo_info)
        if "language" not in repo_info:
            raise Exception(
                "'language' key not round in response\n{}".format(json.dumps(repo_info))
            )
        return repo_info["language"]
    raise Exception(
        f"Expecting a dictionary response from {url}, instead got {json.dumps(repo_info)}"
    )


def get_repo_contents(repo: str) -> List[Dict[str, str]]:
    url = f"https://api.github.com/repos/{repo}/contents/"
    contents = github_api_request(url)
    if type(contents) is list:
        contents = cast(List, contents)
        return contents
    raise Exception(
        f"Expecting a list response from {url}, instead got {json.dumps(contents)}"
    )


def get_readme_download_url(files: List[Dict[str, str]]) -> str:
    """
    Takes in a response from the github api that lists the files in a repo and
    returns the url that can be used to download the repo's README file.
    """
    for file in files:
        if file["name"].lower().startswith("readme"):
            return file["download_url"]
    return ""


def process_repo(repo: str) -> Dict[str, str]:
    """
    Takes a repo name like "gocodeup/codeup-setup-script" and returns a
    dictionary with the language of the repo and the readme contents.
    """
    contents = get_repo_contents(repo)
    readme_download_url = get_readme_download_url(contents)
    if readme_download_url == "":
        readme_contents = ""
    else:
        readme_contents = requests.get(readme_download_url).text
    return {
        "repo": repo,
        "language": get_repo_language(repo),
        "readme_contents": readme_contents,
    }


def scrape_github_data() -> List[Dict[str, str]]:
    """
    Loop through all of the repos and process them. Returns the processed data.
    """
    return [process_repo(repo) for repo in REPOS]


if __name__ == "__main__":
    data = scrape_github_data()
    json.dump(data, open("data.json", "w"), indent=1)

In [11]:
scraped_dict = scrape_github_data()

In [13]:
scraped_df = pd.DataFrame(scraped_dict)
scraped_df

,repo,language,readme_contents
0,MinecraftForge/MinecraftForge,Java,
1,fogleman/Minecraft,Python,# Minecraft\n\nSimple Minecraft-inspired demo ...
2,itzg/docker-minecraft-server,Shell,[![Docker Pulls](https://img.shields.io/docker...
3,overviewer/Minecraft-Overviewer,Python,====================================\nMinecraf...
4,TheGreyGhost/MinecraftByExample,Java,MinecraftByExample [1.16.4]\n=================...
5,minecraft-dev/MinecraftDev,Kotlin,"<p align=""center""><a href=""https://minecraftde..."
6,jdah/minecraft-weekend,C,"# Minecraft, but I made it in 48 hours*\n\n\* ..."
7,mouse0w0/MinecraftDeveloperGuide,None,# 我的世界开发者中文指南\n![](https://img.shields.io/badg...
8,Bukkit/Bukkit,Java,Bukkit\n======\n\nA Minecraft Server API.\n\nW...
9,TheRemote/MinecraftBedrockServer,Shell,# Minecraft Bedrock Server\n\n[日本語版 README はこち...


In [17]:
df = pd.read_csv(r'scraped_data.csv', index_col=[0])
df

,repo,language,readme_contents
0,MinecraftForge/MinecraftForge,Java,NaN
1,fogleman/Minecraft,Python,# Minecraft\n\nSimple Minecraft-inspired demo ...
2,itzg/docker-minecraft-server,Shell,[![Docker Pulls](https://img.shields.io/docker...
3,overviewer/Minecraft-Overviewer,Python,====================================\nMinecraf...
4,TheGreyGhost/MinecraftByExample,Java,MinecraftByExample [1.16.4]\n=================...
...,...,...,...
495,GoLangMc/minecraft-server,Go,"\n<p align=""center"">\n\n <a href=""https://git..."
496,Poudingue/Vanilla-Normals-Renewed-1.12,NaN,# Vanilla-Normals-Renewed-1.12\n\nThis is an a...
497,NiclasOlofsson/MiNET,C#,In case you don't find the information you are...
498,manicdigger/manicdigger,C#,[![Travis Build Status](https://img.shields.io...


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 440 entries, 1 to 499
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   repo             440 non-null    object
 1   language         440 non-null    object
 2   readme_contents  440 non-null    object
dtypes: object(3)
memory usage: 13.8+ KB


In [26]:
df = df.dropna()

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 440 entries, 1 to 499
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   repo             440 non-null    object
 1   language         440 non-null    object
 2   readme_contents  440 non-null    object
dtypes: object(3)
memory usage: 13.8+ KB


In [28]:
df.language.value_counts()

Java                198
Python               41
JavaScript           36
C#                   26
PHP                  18
Shell                18
C++                  17
C                    12
Kotlin               11
TypeScript           11
Go                    9
Rust                  8
Lua                   4
Batchfile             3
Scala                 3
HCL                   2
HTML                  2
CoffeeScript          2
Dockerfile            2
Ruby                  2
TeX                   2
Jupyter Notebook      2
mcfunction            1
Assembly              1
Yacc                  1
Mustache              1
Smarty                1
PureBasic             1
Clojure               1
CSS                   1
PowerShell            1
SystemVerilog         1
ZenScript             1
Name: language, dtype: int64

In [30]:
df.head(51:101)

SyntaxError: invalid syntax (3406975882.py, line 1)

In [33]:
language_list = ['Java','Python','JavaScript']

In [34]:
df = df[df.language.isin(language_list) == True]

In [36]:
df.language.value_counts()

Java          198
Python         41
JavaScript     36
Name: language, dtype: int64

In [37]:
def remove_languages(df):
    #define values
    language_list = ['Java','Python','JavaScript']
    #drop rows that contain any value in the list
    df = df[df.language.isin(language_list) == True]
    
    return df

# Prepare

In [61]:
df = pd.read_csv(r'large_scraped_data.csv', index_col=[0])
df

,repo,language,readme_contents
0,MinecraftForge/MinecraftForge,Java,NaN
1,fogleman/Minecraft,Python,# Minecraft\n\nSimple Minecraft-inspired demo ...
2,itzg/docker-minecraft-server,Shell,[![Docker Pulls](https://img.shields.io/docker...
3,overviewer/Minecraft-Overviewer,Python,====================================\nMinecraf...
4,TheGreyGhost/MinecraftByExample,Java,MinecraftByExample [1.16.4]\n=================...
...,...,...,...
995,GeyserMC/PacketLib,Java,# PacketLib\nPacketLib is a library for packet...
996,Nic4Las/Minecraft-Enderite-Mod,Java,# Enderite Mod\n\n[![](http://cf.way2muchnoise...
997,grahamedgecombe/minecraft-installer,Groff,NaN
998,espertus/blockly-lua,JavaScript,Blockly Lua allows you to program ComputerCraf...


In [62]:
# Replace a whitespace sequence or empty with a NaN value and reassign this manipulation to df. 
df = df.replace(r'^\s*$', np.nan, regex=True)

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   repo             1000 non-null   object
 1   language         938 non-null    object
 2   readme_contents  943 non-null    object
dtypes: object(3)
memory usage: 31.2+ KB


In [64]:
# Removes null values
df = df.dropna()

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 884 entries, 1 to 999
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   repo             884 non-null    object
 1   language         884 non-null    object
 2   readme_contents  884 non-null    object
dtypes: object(3)
memory usage: 27.6+ KB


In [66]:
df.language.value_counts()

Java                 402
JavaScript            86
Python                76
C#                    43
C++                   36
Shell                 35
PHP                   33
Kotlin                23
TypeScript            22
Go                    19
C                     16
Rust                  15
Lua                    6
HTML                   6
ZenScript              5
Ruby                   5
CSS                    4
GLSL                   4
Dockerfile             4
Scala                  4
Jupyter Notebook       4
Batchfile              3
CoffeeScript           3
HCL                    3
Elixir                 2
mcfunction             2
Visual Basic .NET      2
TeX                    2
Mustache               2
PowerShell             2
GDScript               1
R                      1
CMake                  1
Vue                    1
AutoHotkey             1
SCSS                   1
PureBasic              1
Swift                  1
JSON                   1
Assembly               1


In [67]:
#Lowercase everything
df['readme_contents'] = df['readme_contents'].str.lower()

In [68]:
df.head(2)

,repo,language,readme_contents
1,fogleman/Minecraft,Python,# minecraft\n\nsimple minecraft-inspired demo ...
2,itzg/docker-minecraft-server,Shell,[![docker pulls](https://img.shields.io/docker...


In [69]:
#df['readme_contents'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

1      # minecraft\n\nsimple minecraft-inspired demo ...
2      [![docker pulls](https://img.shields.io/docker...
3      ====================================\nminecraf...
4      minecraftbyexample [1.16.4]\n=================...
5      <p align="center"><a href="https://minecraftde...
                             ...                        
994    # crossroads\n\na steampunk technology mod for...
995    # packetlib\npacketlib is a library for packet...
996    # enderite mod\n\n[![](http://cf.way2muchnoise...
998    blockly lua allows you to program computercraf...
999    # node-minecraft-ping\n\nsends a [server list ...
Name: readme_contents, Length: 884, dtype: object

In [71]:
df.head(2)

,repo,language,readme_contents
1,fogleman/Minecraft,Python,# minecraft\n\nsimple minecraft-inspired demo ...
2,itzg/docker-minecraft-server,Shell,[![docker pulls](https://img.shields.io/docker...


In [72]:
#df['readme_plain'] = 
df['readme_contents'].apply(lambda x: markdown.markdown(x))

1      <h1>minecraft</h1>\n<p>simple minecraft-inspir...
2      <p><a href="https://hub.docker.com/r/itzg/mine...
3      <p>====================================\nminec...
4      <h1>minecraftbyexample [1.16.4]</h1>\n<p>the p...
5      <p align="center"><a href="https://minecraftde...
                             ...                        
994    <h1>crossroads</h1>\n<p>a steampunk technology...
995    <h1>packetlib</h1>\n<p>packetlib is a library ...
996    <h1>enderite mod</h1>\n<p><a href="https://www...
998    <p>blockly lua allows you to program computerc...
999    <h1>node-minecraft-ping</h1>\n<p>sends a <a hr...
Name: readme_contents, Length: 884, dtype: object

# Explore

In [ ]:
#Basic clean removes ASCII characters

In [126]:
df = pd.read_csv(r'clean_scraped_data.csv', index_col=[0])
df.head()

,repo,language,readme_contents,clean,lemmatized
1,fogleman/Minecraft,Python,# Minecraft\n\nSimple Minecraft-inspired demo ...,minecraft simple minecraftinspired demo writte...,minecraft simple minecraftinspired demo writte...
2,itzg/docker-minecraft-server,Shell,[![Docker Pulls](https://img.shields.io/docker...,docker pullshttpsimgshieldsiodockerpullsitzgmi...,docker pullshttpsimgshieldsiodockerpullsitzgmi...
3,overviewer/Minecraft-Overviewer,Python,====================================\nMinecraf...,minecraft overviewer build status andrew brown...,minecraft overviewer build status andrew brown...
4,TheGreyGhost/MinecraftByExample,Java,MinecraftByExample [1.16.4]\n=================...,minecraftbyexample 1164 purpose minecraftbyexa...,minecraftbyexample 1164 purpose minecraftbyexa...
5,minecraft-dev/MinecraftDev,Kotlin,"<p align=""center""><a href=""https://minecraftde...",p aligncentera hrefhttpsminecraftdevorgimg src...,p aligncentera hrefhttpsminecraftdevorgimg src...


In [78]:
df.language.value_counts()

Java                 402
JavaScript            86
Python                76
C#                    43
C++                   36
Shell                 35
PHP                   33
Kotlin                23
TypeScript            22
Go                    19
C                     16
Rust                  15
Lua                    6
HTML                   6
ZenScript              5
Ruby                   5
CSS                    4
GLSL                   4
Dockerfile             4
Scala                  4
Jupyter Notebook       4
Batchfile              3
CoffeeScript           3
HCL                    3
Elixir                 2
mcfunction             2
Visual Basic .NET      2
TeX                    2
Mustache               2
PowerShell             2
GDScript               1
R                      1
CMake                  1
Vue                    1
AutoHotkey             1
SCSS                   1
PureBasic              1
Swift                  1
JSON                   1
Assembly               1


In [94]:
df.head(2)

,repo,language,readme_contents,clean,lemmatized
1,fogleman/Minecraft,Python,# Minecraft\n\nSimple Minecraft-inspired demo ...,minecraft simple minecraftinspired demo writte...,minecraft simple minecraftinspired demo writte...
2,itzg/docker-minecraft-server,Shell,[![Docker Pulls](https://img.shields.io/docker...,docker pullshttpsimgshieldsiodockerpullsitzgmi...,docker pullshttpsimgshieldsiodockerpullsitzgmi...


In [127]:
language_list = df['language'].unique().tolist()
language_list.remove("JavaScript")
language_list.remove("Python")
language_list.remove("Java")

In [128]:
language_list

['Shell',
 'Kotlin',
 'C',
 'C#',
 'Lua',
 'C++',
 'PHP',
 'TypeScript',
 'SystemVerilog',
 'Scala',
 'PowerShell',
 'CSS',
 'Batchfile',
 'Mustache',
 'Rust',
 'Go',
 'CoffeeScript',
 'Dockerfile',
 'Clojure',
 'PureBasic',
 'TeX',
 'Jupyter Notebook',
 'Smarty',
 'HTML',
 'Yacc',
 'Ruby',
 'mcfunction',
 'HCL',
 'Assembly',
 'ZenScript',
 'GLSL',
 'JSON',
 'Swift',
 'Vue',
 'SCSS',
 'AutoHotkey',
 'Visual Basic .NET',
 'Elixir',
 'GDScript',
 'CMake',
 'R',
 'Haskell']

In [124]:
for i in range(len(language_list)):
    var = language_list[i]
    df['language'] = df['language'].map({var:'other'})

In [125]:
df.language.value_counts()

Series([], Name: language, dtype: int64)

In [129]:
top_languages = ["Python", "Java", "JavaScript"]

In [ ]:
for df[~df['language'].isin(top_languages)]:
    df['language'].replace()

In [ ]:
#if language in df.language_list != top_languages:
 #   df['language'].map

In [157]:
df.loc[~df['language'].isin(top_languages), 'language'] = 'Other'


In [158]:
df

,repo,language,readme_contents,clean,lemmatized,is_other
1,fogleman/Minecraft,Python,# Minecraft\n\nSimple Minecraft-inspired demo ...,minecraft simple minecraftinspired demo writte...,minecraft simple minecraftinspired demo writte...,False
2,itzg/docker-minecraft-server,Other,[![Docker Pulls](https://img.shields.io/docker...,docker pullshttpsimgshieldsiodockerpullsitzgmi...,docker pullshttpsimgshieldsiodockerpullsitzgmi...,True
3,overviewer/Minecraft-Overviewer,Python,====================================\nMinecraf...,minecraft overviewer build status andrew brown...,minecraft overviewer build status andrew brown...,False
4,TheGreyGhost/MinecraftByExample,Java,MinecraftByExample [1.16.4]\n=================...,minecraftbyexample 1164 purpose minecraftbyexa...,minecraftbyexample 1164 purpose minecraftbyexa...,False
5,minecraft-dev/MinecraftDev,Other,"<p align=""center""><a href=""https://minecraftde...",p aligncentera hrefhttpsminecraftdevorgimg src...,p aligncentera hrefhttpsminecraftdevorgimg src...,True
...,...,...,...,...,...,...
994,Crossroads-Development/Crossroads,Java,# Crossroads\n\nA steampunk technology mod for...,crossroads steampunk technology mod minecraft ...,crossroad steampunk technology mod minecraft i...,False
995,GeyserMC/PacketLib,Java,# PacketLib\nPacketLib is a library for packet...,packetlib packetlib library packetbased networ...,packetlib packetlib library packetbased networ...,False
996,Nic4Las/Minecraft-Enderite-Mod,Java,# Enderite Mod\n\n[![](http://cf.way2muchnoise...,enderite mod httpcfway2muchnoiseeuversions3992...,enderite mod httpcfway2muchnoiseeuversions3992...,False
998,espertus/blockly-lua,JavaScript,Blockly Lua allows you to program ComputerCraf...,blockly lua allows program computercraft turtl...,blockly lua allows program computercraft turtl...,False
